In [1]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

#define the bitcoin api url 
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'5000',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '5660fbe3-b95f-41cc-a749-5f328b7cae7e',
}
#this api key given to users who sign up will direct you to bitcoin price data
#using session we go through to get and convert to json file
session = Session()
session.headers.update(headers)

In [2]:
#so this sends an HTTP GET request to the URL and saves the response from API to JSON
#include exception error to catch any
#this will return a dictionary
try:
  response = session.get(url, params=parameters)
  response_json = json.loads(response.text)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

dict

In [3]:
#deleting the unwanted information as it clouds the dataset
#for this project we just need USD current prices which is in data
del response_json['status']

In [4]:
dataList=[]
#convert the data dict to a list
dataList=response_json['data']

In [5]:
#this function will then extract just the latest USD price
def get_latest_bitcoin_price():
    return float(dataList[0]['quote']['USD']['price'])

In [6]:
#here we create the notifications on ifttt creating
ifttt_webhook_url='https://maker.ifttt.com/trigger/{}/with/key/dQ0Uu6Kqk_Q948N_Trt_uKj9YgewlErluSodGAtksr2'
#this function takes in event and value and puts it into session to trigger the notification
def post_ifttt_webhook(event,value):
    data={'value1' : value}
    ifttt_event_url=ifttt_webhook_url.format(event)
    session.post(ifttt_event_url,json=data)
    

In [7]:
#this formats the notification result
def format_bitcoin_history(bitcoin_history):
    rows = []
    for bitcoin_price in bitcoin_history:
        date = bitcoin_price['date'].strftime('%d.%m.%Y %H:%M')
        price = bitcoin_price['price']
        row = '{}: $<b>{}</b>'.format(date, price)
        rows.append(row)

    return '<br>'.join(rows)

In [8]:
import requests
import time
from datetime import datetime

threshold=10000


def main():
    bitcoin_history = []
    while True:
        price = get_latest_bitcoin_price()
        date = datetime.now()
        bitcoin_history.append({'date': date, 'price': price})
#sends notification when the price falls below threshold it urges user to buy
        if price < threshold:
            post_ifttt_webhook('bitcoin_price_emergency', price)
#sends normal price updates
        if len(bitcoin_history) == 5:
            post_ifttt_webhook('bitcoin_price_update', format_bitcoin_history(bitcoin_history))
            bitcoin_history = []
            
        time.sleep(1 * 60)
if __name__ == '__main__':
    main()

ConnectionError: ('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None))